In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

In [2]:
def preprocess(x, y) : 
    # [0 ~ 255] => [-1 ~ 1]
    x = 2 * tf.cast(x, dtype = tf.float32) / 255. - 1.
    y = tf.cast(y, dtype = tf.int32)
    
    return x, y

In [3]:
batch_size = 128

# [50k, 32, 32, 3], [10k, 1]
(x, y), (x_test, y_test) = datasets.cifar10.load_data()

y = tf.squeeze(y)
y_test = tf.squeeze(y_test)

y = tf.one_hot(y, depth = 10) # [50k, 10]
y_test = tf.one_hot(y_test, depth = 10) # [10k, 10]

print('dataset : ', x.shape, y.shape, x.min(), x.max())

dataset :  (50000, 32, 32, 3) (50000, 10) 0 255


In [4]:
train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batch_size)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(batch_size)

In [5]:
sample = next(iter(train_db))

print(sample[0].shape, sample[1].shape)

(128, 32, 32, 3) (128, 10)


In [6]:
class MyDense(layers.Layer) :
    # to replace standard layers.Dense()
    def __init__(self, in_dim, out_dim) :
        super(MyDense, self).__init__()
        
        self.kernel = self.add_variable('W', [in_dim, out_dim])
        # self.bias = self.add_variable('b', [out_dim])
        
    def call(self, inputs, tarining = None) :
        
        out = inputs @ self.kernel # + self.bias
        
        return out

In [7]:
class MyNetwork(keras.Model) :
    
    def __init__(self) :
        super(MyNetwork, self).__init__()
        
        self.fc1 = MyDense(32*32*3, 1028)
        self.fc2 = MyDense(1028, 512)
        self.fc3 = MyDense(512, 256)
        self.fc4 = MyDense(256, 128)
        self.fc5 = MyDense(128, 64)
        self.fc6 = MyDense(64, 32)
        self.fc7 = MyDense(32, 16)
        self.fc8 = MyDense(16, 10)
        
    def call(self, inputs, tarining = None) : 
        """
        parameters => inputs : [b, 32, 32, 3]
        parameters => training : 
        return : 
        """
        x = tf.reshape(inputs, [-1, 32*32*3])
        # [b, 32*32*3] => [b, 256]
        x = tf.nn.relu(self.fc1(x))
        # [b, 256] => [b, 128]
        x = tf.nn.relu(self.fc2(x))
        x = tf.nn.relu(self.fc3(x))
        x = tf.nn.relu(self.fc4(x))
        x = tf.nn.relu(self.fc5(x))
        x = tf.nn.relu(self.fc6(x))
        x = tf.nn.relu(self.fc7(x))
        x = self.fc8(x)
        
        return x

In [8]:
network = MyNetwork()

C:\Anaconda\envs\MyFirstTensorflow\lib\site-packages\keras\engine\base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [9]:
# from_logits = True : https://www.cnblogs.com/harrysong666/p/14635131.html
network.compile(optimizer = optimizers.Adam(learning_rate = 1e-3),
               loss = tf.losses.CategoricalCrossentropy(from_logits = True),
               metrics = ['accuracy'])

network.fit(train_db, epochs = 15, validation_data = test_db, validation_freq = 2)

Epoch 1/15
391/391 [==============================] - 7s 8ms/step - loss: 1.8287 - accuracy: 0.3544
Epoch 2/15
391/391 [==============================] - 3s 8ms/step - loss: 1.5464 - accuracy: 0.4563 - val_loss: 1.5151 - val_accuracy: 0.4672
Epoch 3/15
391/391 [==============================] - 3s 7ms/step - loss: 1.4236 - accuracy: 0.5003
Epoch 4/15
391/391 [==============================] - 3s 8ms/step - loss: 1.3254 - accuracy: 0.5367 - val_loss: 1.3983 - val_accuracy: 0.5134
Epoch 5/15
391/391 [==============================] - 2s 6ms/step - loss: 1.2368 - accuracy: 0.5659
Epoch 6/15
391/391 [==============================] - 3s 7ms/step - loss: 1.1550 - accuracy: 0.5942 - val_loss: 1.4164 - val_accuracy: 0.5232
Epoch 7/15
391/391 [==============================] - 2s 6ms/step - loss: 1.0750 - accuracy: 0.6198
Epoch 8/15
391/391 [==============================] - 3s 7ms/step - loss: 0.9995 - accuracy: 0.6490 - val_loss: 1.4127 - val_accuracy: 0.5415
Epoch 9/15
391/391 [============

In [10]:
network.evaluate(test_db)

network.save_weights('mySaveWeight/weights.ckpt')
del network

print('saved to mySaveWeight/weights.ckpt')

79/79 [==============================] - 0s 3ms/step - loss: 1.9411 - accuracy: 0.5378
saved to mySaveWeight/weights.ckpt


In [11]:
# loaded weights
network = MyNetwork()

network.compile(optimizer = optimizers.Adam(learning_rate = 1e-3),
               loss = tf.losses.CategoricalCrossentropy(from_logits = True),
               metrics = ['accuracy'])

network.load_weights('mySaveWeight/weights.ckpt')

print('loaded weights from mySaveWeight/weights.ckpt')

network.evaluate(test_db)

loaded weights from mySaveWeight/weights.ckpt
79/79 [==============================] - 0s 3ms/step - loss: 1.9411 - accuracy: 0.5378


[1.9411288499832153, 0.5378000140190125]

# Reference
# 网易云课堂《深度学习与TensorFlow 2入门实战》
https://www.youtube.com/playlist?list=PLh7DRwYmUgh7swOvZUZ52LMeGDmjFH0nv